In [1]:
import imageio
import h5py
import numpy as np

In [2]:
samples = ['s1']
p_noisy = ['00','10','15','25'] # # 0% is GT (clean) and the rest are noisy inputs
frame_count = 279 # number of frames per case

In [3]:
#loaded_data = None
for s in samples:
    loaded_sample = None
    for p in p_noisy:
        loaded_frame = None
        for i in range(frame_count):
            for lr in [0,1]:
                file_name = 'dataset/cropped/frames_split_1024/'+s+'_'+p+'_'+str(i+1).zfill(3)+'-'+str(lr)+'.jpg'
                #print(file_name)
                im = imageio.imread(file_name)
                np_im = np.array(im)
                np_im = np.expand_dims(np_im,axis = 0)
                if (loaded_frame is None):
                    loaded_frame = np_im
                else:
                    #print(np_im.shape,loaded_frame.shape)
                    loaded_frame = np.append(loaded_frame,np_im,axis=0)
        loaded_frame = np.expand_dims(loaded_frame,axis = 0)
        print('frames: ',loaded_frame.shape)
        hf = h5py.File('dataset/cropped/data_'+s+'_'+p+'_1024x1024.h5', 'w') # (frame,noise,width,height,channel)
        hf.create_dataset('images', data=loaded_frame)
        hf.close()
    #if (loaded_filter is None):
    #    loaded_filter = loaded_frame
    #else:
    #    loaded_filter = np.append(loaded_filter,loaded_frame,axis=0)
    """    
    loaded_filter = np.expand_dims(loaded_filter,axis = 0)
    print('filters: ',loaded_filter.shape)
    hf = h5py.File('dataset/cropped/data_'+s+'_'+p+'_1024x1024.h5', 'w') # (frame,noise,width,height,channel)
    hf.create_dataset('images', data=loaded_sample) 
    hf.close()
    """
    #if (loaded_sample is None):
    #    loaded_sample = loaded_filter
    #else:
    #    loaded_sample = np.append(loaded_sample,loaded_filter,axis=0)
    #print(loaded_sample.shape)
    #loaded_sample = np.expand_dims(loaded_sample,axis = 0)
    #hf = h5py.File('dataset/cropped/data_'+s+'_1024x1024.h5', 'w') # (frame,noise,width,height,channel)
    #hf.create_dataset('images', data=loaded_sample) 
    #hf.close()

frames:  (1, 558, 1024, 1024, 3)
frames:  (1, 558, 1024, 1024, 3)
frames:  (1, 558, 1024, 1024, 3)
frames:  (1, 558, 1024, 1024, 3)


In [4]:
# train and test data only based on samples s1
train_split = 0.8
train_data = None
test_data = None
frame_count = 558 # split
for s in ['s1']: # ignore s2,s3 for now
    frame_range = np.arange(frame_count) # frame indices for all samples
    np.random.shuffle(frame_range)
    train_range = frame_range[:int(frame_count *  train_split)]
    train_range = np.sort(train_range)
    test_range = frame_range[int(frame_count *  train_split):]
    test_range = np.sort(test_range)
    for n in ['00','10','15','25']:
        f = h5py.File("dataset/cropped/h5/data_"+s+"_"+n+"_1024x1024.h5", "r")
        all_data = f['images']
        train_data = np.squeeze(all_data, axis=0)[train_range,:,:,0]
        test_data = np.squeeze(all_data, axis=0)[test_range,:,:,0]
        #all_data = np.transpose(all_data,(0,3,1,2)) # (frame,noise,channel,height,width)
        print(all_data.shape)
        f.close()
        if n == '00':
            hf = h5py.File('dataset/cropped/h5/clean4train_'+s+'.h5', 'w')
            hf.create_dataset('images', data=train_data)
            hf.close()
            hf = h5py.File('dataset/cropped/h5/clean4test_'+s+'.h5', 'w')
            hf.create_dataset('images', data=test_data)
            hf.close()
        else:
            hf = h5py.File('dataset/cropped/h5/noisy4train_n'+n+'_'+s+'.h5', 'w')
            hf.create_dataset('images', data=train_data)
            hf.close()
            hf = h5py.File('dataset/cropped/h5/noisy4test_n'+n+'_'+s+'.h5', 'w')
            hf.create_dataset('images', data=test_data)
            hf.close()


(1, 558, 1024, 1024, 3)
(1, 558, 1024, 1024, 3)
(1, 558, 1024, 1024, 3)
(1, 558, 1024, 1024, 3)


In [6]:
for tt in ['noisy']:
    for nn in ['10','15','25']:
        #print('dataset/cropped/'+tt+'4train_n'+nn+'_s2.h5')
        f_s1 = h5py.File('dataset/cropped/h5/'+tt+'4train_n'+nn+'_s1.h5', "r")
        f_s2 = h5py.File('dataset/cropped/h5/'+tt+'4train_n'+nn+'_s2.h5', "r")
        f_s3 = h5py.File('dataset/cropped/h5/'+tt+'4train_n'+nn+'_s3.h5', "r")
        s1_data = f_s1['images']
        s2_data = f_s2['images']
        s3_data = f_s3['images']
        s1_s2_data = np.append(s1_data,s2_data,axis=0)
        s1_s2_s3_data = np.append(s1_s2_data,s3_data,axis=0)
        hf = h5py.File('dataset/cropped/h5/'+tt+'4train_n'+nn+'_s1_s2_s3.h5', 'w')
        hf.create_dataset('images', data=s1_s2_s3_data)
        hf.close()
        f_s1.close()
        f_s2.close()
        f_s3.close()
    
for tt in ['clean']:
    print('dataset/cropped/h5/'+tt+'4train_s2.h5')
    f_s1 = h5py.File('dataset/cropped/h5/'+tt+'4train_s1.h5', "r")
    f_s2 = h5py.File('dataset/cropped/h5/'+tt+'4train_s2.h5', "r")
    f_s3 = h5py.File('dataset/cropped/h5/'+tt+'4train_s3.h5', "r")
    s1_data = f_s1['images']
    s2_data = f_s2['images']
    s3_data = f_s3['images']
    s1_s2_data = np.append(s1_data,s2_data,axis=0)
    s1_s2_s3_data = np.append(s1_s2_data,s3_data,axis=0)
    hf = h5py.File('dataset/cropped/h5/'+tt+'4train_s1_s2_s3.h5', 'w')
    hf.create_dataset('images', data=s1_s2_s3_data)
    hf.close()
    f_s1.close()
    f_s2.close()
    f_s3.close()

dataset/cropped/h5/clean4train_s2.h5


In [7]:
for tt in ['noisy']:
    for nn in ['10','15','25']:
        #print('dataset/cropped/'+tt+'4test_n'+nn+'_s2.h5')
        f_s1 = h5py.File('dataset/cropped/h5/'+tt+'4test_n'+nn+'_s1.h5', "r")
        f_s2 = h5py.File('dataset/cropped/h5/'+tt+'4test_n'+nn+'_s2.h5', "r")
        f_s3 = h5py.File('dataset/cropped/h5/'+tt+'4test_n'+nn+'_s3.h5', "r")
        s1_data = f_s1['images']
        s2_data = f_s2['images']
        s3_data = f_s3['images']
        s1_s2_data = np.append(s1_data,s2_data,axis=0)
        s1_s2_s3_data = np.append(s1_s2_data,s3_data,axis=0)
        hf = h5py.File('dataset/cropped/h5/'+tt+'4test_n'+nn+'_s1_s2_s3.h5', 'w')
        hf.create_dataset('images', data=s1_s2_s3_data)
        hf.close()
        f_s1.close()
        f_s2.close()
        f_s3.close()
    
for tt in ['clean']:
    #print('dataset/cropped/'+tt+'4test_s2.h5')
    f_s1 = h5py.File('dataset/cropped/h5/'+tt+'4test_s1.h5', "r")
    f_s2 = h5py.File('dataset/cropped/h5/'+tt+'4test_s2.h5', "r")
    f_s3 = h5py.File('dataset/cropped/h5/'+tt+'4test_s3.h5', "r")
    s1_data = f_s1['images']
    s2_data = f_s2['images']
    s3_data = f_s3['images']
    s1_s2_data = np.append(s1_data,s2_data,axis=0)
    s1_s2_s3_data = np.append(s1_s2_data,s3_data,axis=0)
    hf = h5py.File('dataset/cropped/h5/'+tt+'4test_s1_s2_s3.h5', 'w')
    hf.create_dataset('images', data=s1_s2_s3_data)
    hf.close()
    f_s1.close()
    f_s2.close()
    f_s3.close()